In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt

In [2]:
# RFE feature selection function
def rfeFeature(indep_X, dep_Y, n):
    rfelist = []
    from sklearn.linear_model import LinearRegression
    lin = LinearRegression()
    from sklearn.svm import SVR
    SVRl = SVR(kernel='linear')
    from sklearn.svm import SVR
    # SVRnl = SVR(kernel='rbf')
    from sklearn.tree import DecisionTreeRegressor
    dec = DecisionTreeRegressor(random_state=0)
    from sklearn.ensemble import RandomForestRegressor
    rf = RandomForestRegressor(n_estimators=10, random_state=0)
    rfemodellist = [lin, SVRl, dec, rf]
    for i in rfemodellist:
        print(i)
        log_rfe = RFE(i, n_features_to_select=n)
        log_fit = log_rfe.fit(indep_X, dep_Y)
        log_rfe_feature = log_fit.transform(indep_X)
        rfelist.append(log_rfe_feature)
    return rfelist


In [3]:
# Function to split and scale data
def split_scalar(indep_X, dep_Y):
    X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size=0.25, random_state=0)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    return X_train, X_test, y_train, y_test

# Function to calculate R-squared
def r2_prediction(regressor, X_test, y_test):
    y_pred = regressor.predict(X_test)
    from sklearn.metrics import r2_score
    r2 = r2_score(y_test, y_pred)
    return r2

# Regression functions
def Linear(X_train, y_train, X_test):
    from sklearn.linear_model import LinearRegression
    regressor = LinearRegression()
    regressor.fit(X_train, y_train)
    r2 = r2_prediction(regressor, X_test, y_test)
    return r2

def svm_linear(X_train, y_train, X_test):
    from sklearn.svm import SVR
    regressor = SVR(kernel='linear')
    regressor.fit(X_train, y_train)
    r2 = r2_prediction(regressor, X_test, y_test)
    return r2

def svm_NL(X_train, y_train, X_test):
    from sklearn.svm import SVR
    regressor = SVR(kernel='rbf')
    regressor.fit(X_train, y_train)
    r2 = r2_prediction(regressor, X_test, y_test)
    return r2

def Decision(X_train, y_train, X_test):
    from sklearn.tree import DecisionTreeRegressor
    regressor = DecisionTreeRegressor(random_state=0)
    regressor.fit(X_train, y_train)
    r2 = r2_prediction(regressor, X_test, y_test)
    return r2

def random(X_train, y_train, X_test):
    from sklearn.ensemble import RandomForestRegressor
    regressor = RandomForestRegressor(n_estimators=10, random_state=0)
    regressor.fit(X_train, y_train)
    r2 = r2_prediction(regressor, X_test, y_test)
    return r2

In [4]:
# RFE regression function
def rfe_regression(acclog, accsvml, accdes, accrf):
    data = {
        'Linear': acclog,
        'SVMl': accsvml,
        'Decision': accdes,
        'Random': accrf
    }
    rfedataframe = pd.DataFrame(data)
    return rfedataframe


In [13]:
# Load dataset
dataset1 = pd.read_csv("DS_jOBs.csv", index_col=None)
df2 = dataset1
df2 = pd.get_dummies(df2, drop_first=True)

# Split data into independent (X) and dependent (Y) variables
indep_X = df2.drop('salary', axis=1)
dep_Y = df2['salary']

# Perform RFE feature selection
rfelist = rfeFeature(indep_X, dep_Y, 7)

LinearRegression()
SVR(kernel='linear')
DecisionTreeRegressor(random_state=0)
RandomForestRegressor(n_estimators=10, random_state=0)


In [14]:
# Initialize accuracy lists
acclin = []
accsvml = []
accsvmnl = []
accdes = []
accrf = []


In [15]:

# Train and evaluate models
for features in rfelist:
    X_train, X_test, y_train, y_test = train_test_split(features, dep_Y, test_size=0.25, random_state=0)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    r2_lin = Linear(X_train, y_train, X_test)
    acclin.append(r2_lin)
    
    r2_sl = svm_linear(X_train, y_train, X_test)
    accsvml.append(r2_sl)
    
    r2_d = Decision(X_train, y_train, X_test)
    accdes.append(r2_d)
    
    r2_r = random(X_train, y_train, X_test)
    accrf.append(r2_r)

    
result=rfe_regression(acclin,accsvml,accdes,accrf)

In [16]:
#7
result

,Linear,SVMl,Decision,Random
0,0.138505,-0.018088,0.234995,0.235371
1,0.120875,-0.016233,0.200015,0.199626
2,0.114615,-0.019097,-0.681063,-0.054339
3,0.120281,-0.019310,-0.680400,-0.074606


In [ ]:
#Random
#0.235371

In [12]:
#10
result

,Linear,SVMl,Decision,Random
0,0.140216,-0.017801,0.227201,0.229650
1,0.122275,-0.015709,0.181073,0.182345
2,0.114701,-0.019009,-0.764443,-0.052569
3,0.114821,-0.019269,-0.705290,-0.048197
